In [25]:
%%capture --no-stderr
%pip install pyautogen~=0.2.0b4

In [8]:
import autogen

config_list_gpt4 = autogen.config_list_from_json("OAI_CONFIG_LIST")

In [11]:
gpt4_config = {
    "cache_seed": 52,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list_gpt4,
    "timeout": 120,
    "max_tokens": 4000,
}
user_proxy = autogen.UserProxyAgent(
   name="Boss",
   system_message="""Boss.  Discuss the current proposed plan with Planner, Product Manager, Engineer, CodeSaver, and Executor. The execution of the plan requires approval from the boss.
   """,
   code_execution_config=False,
)
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=gpt4_config,
    system_message="""Engineer. follow the approved plan and write language code such as typescript, nodejs, and HTML to solve the task. You are an engineer with high engineering literacy, and you will tell CodeSaver to place the corresponding requirement code in different folders and files.
The user is unable to modify your code. The code you replied to is ultimately executed by the Executor. In order to avoid unnecessary errors and corrections, it is not recommended that incomplete code needs to be modified or supplemented by others. The replied code block must be complete, not partial or pseudocode. If the code block is not intended to be executed by the Executor, do not use it.
The code for another project is likely to be a combination of multiple code files, so after you finish writing one code file, ask CodeSaver to save it and then write the next code file. Remember to ensure that each communication with CodeSaver only includes the content of one code file to be stored by CodeSaver. After all the files have been written and CodeSaver has been saved, you can tell Executor to execute your code.
Check the execution result returned by the Executor. If the result indicates an error, please fix the error and output the code again. If the error cannot be fixed, or if the task is not resolved even after the code is successfully executed, analyze the problem, re-examine your hypothesis, collect additional information needed, and come up with different methods to try.
""",
)

planner = autogen.AssistantAgent(
    name="Planner",
    system_message="""Planner. Draw up a plan for the current task. Revise the plan based on feedback from Boss, Product Manager, Engineer, CodeSaver, and Executor until approved by Boss.
This plan may involve an Engineer who can write code, a CodeSaver who cannot execute but only saves code, and an Executor who executes code at the end.
Firstly, explain the plan. Clearly identify which step was completed by the Engineer, which step was completed by CodeSaver, and which step was completed by the Executor.""",
    llm_config=gpt4_config,
)
executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. After saving each code file locally in CodeSaver, execute the code written by the Engineer and report the results.",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": 3, "work_dir": "site", "use_docker": "false"},
    # llm_config=gpt4_config,
)
code = autogen.UserProxyAgent(
    name="CodeSaver",
    system_message="CodeSaver. Don't execute code, only store files of different codes according to the directory structure provided by the engineer, based on the code received by the engineer.",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": 3, "work_dir": "site", "use_docker": "false"},
    # llm_config=gpt4_config,
)


pm = autogen.AssistantAgent(
    name="Product-Manager",
    system_message="Product Manager. Carefully check the behavior of CodeSaver, do not let him execute the code, and provide feedback. Need to ensure that Boss's requirements are truly completed",
    llm_config=gpt4_config,
)
groupchat = autogen.GroupChat(agents=[user_proxy, engineer, planner, executor, pm, code], messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)


In [12]:
user_proxy.initiate_chat(
    manager,
    message="""
Write a web version of the Snake game that requires NodeJS to start and run. Non code related content does not need to be considered temporarily
""",
)

Boss (to chat_manager):


写一个 Web 版的贪吃蛇游戏，需要用NodeJS 启动运行。非代码相关的内容暂不需要考虑


--------------------------------------------------------------------------------
Planner (to chat_manager):

为了完成Boss的任务，我们需要制定一个详细的计划来开发一个Web版的贪吃蛇游戏，并使用NodeJS来启动运行。以下是计划的步骤，以及每个角色的职责：

1. **需求分析和设计**:
   - **Product Manager**: 定义游戏的功能需求，如游戏规则、用户界面、得分系统等，并与Boss确认。
   - **Engineer**: 参与需求讨论，提供技术可行性建议。

2. **技术选型和环境搭建**:
   - **Engineer**: 确定使用的技术栈（如Node.js, Express, Socket.IO等），搭建开发环境。

3. **编码**:
   - **Engineer**: 编写游戏的前端代码（HTML, CSS, JavaScript）和后端代码（Node.js）。
   - **CodeSaver**: 在Engineer完成代码编写后，负责保存和版本控制。

4. **代码审查**:
   - **Engineer**: 对编写的代码进行自我审查。
   - **Product Manager**: 确认代码是否满足需求。
   - **Boss**: 最终审批代码。

5. **测试**:
   - **Engineer**: 进行单元测试和集成测试，确保游戏运行无误。
   - **Product Manager**: 进行用户测试，确保游戏体验符合预期。

6. **部署**:
   - **Engineer**: 将代码部署到服务器。
   - **CodeSaver**: 确保部署的代码版本正确。

7. **运行和维护**:
   - **Executor**: 负责启动Node.js服务器，运行游戏。
   - **Engineer**: 监控游戏运行情况，进行必要的维护和更新。

8. **反馈和迭代**:
   - **Product Manag